## 此 install 是用來debug的。install 過後，要重新啟動工作階段才能運行成功

In [69]:
!pip install accelerate==0.23.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 5.7 MB/s eta 0:00:00


# 開雲端空間

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 正式用

## <=5

### 賦予變數

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display

# 指定 Parquet 檔案的路徑
train_source_file_path = '/content/drive/MyDrive/Datagame-2023/data/label_train_source.parquet'
train_target_file_path = '/content/drive/MyDrive/Datagame-2023/data/label_train_target.parquet'
test_source_file_path = '/content/drive/MyDrive/Datagame-2023/data/label_test_source.parquet'
meta_song_file_path = '/content/drive/MyDrive/Datagame-2023/data/meta_song.parquet'
song_composer_file_path = '/content/drive/MyDrive/Datagame-2023/data/meta_song_composer.parquet'
song_genre_file_path = '/content/drive/MyDrive/Datagame-2023/data/meta_song_genre.parquet'
song_lyricist_file_path = '/content/drive/MyDrive/Datagame-2023/data/meta_song_lyricist.parquet'
song_producer_file_path = '/content/drive/MyDrive/Datagame-2023/data/meta_song_producer.parquet'
song_titletext_file_path = '/content/drive/MyDrive/Datagame-2023/data/meta_song_titletext.parquet'

train_df = pd.read_parquet(train_source_file_path)

In [3]:
meta_song_df = pd.read_parquet(meta_song_file_path)
# 将 NaN 值替换为 0，并转换为整数
meta_song_df['album_id'] = meta_song_df['album_id'].fillna(0).astype(int)
meta_song_df['artist_id'] = meta_song_df['artist_id'].fillna(0).astype(int)

train_meta_df = pd.merge(train_df, meta_song_df[['song_id', 'artist_id', 'album_id']], on='song_id', how='left')
album_song_count = train_meta_df.groupby('album_id')['song_id'].value_counts()
artist_song_count = train_meta_df.groupby('artist_id')['song_id'].value_counts()
song_count = train_meta_df['song_id'].value_counts()

song_id_mapping = dict(zip(meta_song_df['song_id'], zip(meta_song_df['album_id'], meta_song_df['artist_id'])))

del train_df
del meta_song_df

### 做出每個session_id對應的album_id跟artist_id的播放數 (session_album_artist)

#### 快速版的

In [4]:
unique_counts = train_meta_df.groupby('session_id')['song_id'].nunique()
under_5_counts = unique_counts[unique_counts <= 5].index.tolist()
selected_rows = train_meta_df[train_meta_df['session_id'].isin(under_5_counts)]
session_album_artist = {}
# 遍历每个 session_id 的 song_id
for session_id, song_id in zip(selected_rows['session_id'], selected_rows['song_id']):
    if song_id not in song_id_mapping:
        continue

    # 获取专辑和艺术家
    album_id, artist_id = song_id_mapping[song_id]

    # 获取局部变量以减少嵌套访问
    session_albums = session_album_artist.setdefault(session_id, {'albums': {}})
    session_artists = session_albums.setdefault('artists', {})

    # album_id 非 nan
    if not np.isnan(album_id):
        album_id = int(album_id)
        session_albums['albums'][album_id] = session_albums['albums'].get(album_id, 0) + 1

    # artist_id 非 nan
    if not np.isnan(artist_id):
        artist_id = int(artist_id)
        session_artists[artist_id] = session_artists.get(artist_id, 0) + 1

    session_album_artist[session_id]['albums'] = dict(sorted(session_album_artist[session_id]['albums'].items(), key=lambda x: x[1], reverse=True))
    session_album_artist[session_id]['artists'] = dict(sorted(session_album_artist[session_id]['artists'].items(), key=lambda x: x[1], reverse=True))



#### 慢速版的

In [5]:
# result_dict = {}

# # 遍历每个session_id的song_id
# for session_id, song_id in zip(selected_rows['session_id'][2000:2100], selected_rows['song_id'][2000:2100]):
#     if song_id not in meta_song_df['song_id'].values:
#       # 可以选择忽略该条记录或者采取其他处理方式
#       continue

#     # 找到对应的专辑和艺术家
#     album_id = meta_song_df[meta_song_df['song_id']==song_id]['album_id'].values[0]
#     artist_id = meta_song_df[meta_song_df['song_id']==song_id]['artist_id'].values[0]

#     # 更新字典
#     if session_id not in result_dict:
#         result_dict[session_id] = {'albums': {}, 'artists': {}}

#     # album_id非nan
#     if not np.isnan(album_id):
#       album_id = album_id.astype(int)
#       # 统计专辑数量
#       if album_id not in result_dict[session_id]['albums']:

#           result_dict[session_id]['albums'][album_id] = 1
#       else:
#           result_dict[session_id]['albums'][album_id] += 1

#     # artist_id非nan
#     if not np.isnan(artist_id):
#       artist_id = artist_id.astype(int)
#       # 统计艺术家数量
#       if artist_id not in result_dict[session_id]['artists'] and not np.isnan(artist_id):
#           result_dict[session_id]['artists'][artist_id] = 1
#       else:
#           result_dict[session_id]['artists'][artist_id] += 1

# # 输出结果
# print(result_dict)


### 把所有播放過的歌曲放進去 (session_dict)

In [16]:
selected_rows_sorted = selected_rows.sort_values(['session_id', 'unix_played_at'])
# 使用 drop_duplicates 保留每個 session_id 中相同 song_id 的最後一行
final_selected_rows = selected_rows_sorted.drop_duplicates(['session_id', 'song_id'], keep='last')[['session_id','song_id']]
session_dict = {}

# 遍历数据框的每一行
for index, row in final_selected_rows.iterrows():
    session_id = row['session_id']
    song_id = row['song_id']

    # 如果session_id不在字典中，创建一个新的列表
    if session_id not in session_dict:
        session_dict[session_id] = []

    # 将歌曲添加到session_id对应的列表中
    session_dict[session_id].append(song_id)
    # 创建一个字典，将 song_id 映射到专辑和艺术家

### 填補沒放完5個的空缺

In [17]:
for session_id in session_dict:
  # 有空缺的話
  if len(session_dict[session_id])<5:
    if not session_album_artist[session_id]['albums']=={}:
      albums = session_album_artist[session_id]['albums']
      for album in albums:
        # 此album播放的次數必須大於3，代表此用戶在聽此album
        if albums[album]>3:
          # 抓出專輯裡的所有歌曲由高到低排名
          for song_id in album_song_count.loc[album].index.get_level_values('song_id').tolist():
            if song_id not in session_dict[session_id]:
              session_dict[session_id].append(song_id)
            if len(session_dict[session_id])>=5:
              break
        else:
          break
        if len(session_dict[session_id])>=5:
          break
  else:
    continue
  # 還是有空缺的話
  if len(session_dict[session_id])<5:
    if not session_album_artist[session_id]['artists']=={}:
      artists = session_album_artist[session_id]['artists']
      for artist in artists:
        # 此artists播放的次數必須大於3，代表此用戶在聽此artists
        if artists[artist]>3:
          # 抓出artists的所有歌曲由高到低排名
          for song_id in artist_song_count.loc[artist].index.get_level_values('song_id').tolist():
            if song_id not in session_dict[session_id]:
              session_dict[session_id].append(song_id)
            if len(session_dict[session_id])>=5:
              break
        else:
          break
        if len(session_dict[session_id])>=5:
          break
  else:
    continue
  if len(session_dict[session_id])<5:
    for song_id in song_count.index:
      if song_id not in session_dict[session_id]:
        session_dict[session_id].append(song_id)
      if len(session_dict[session_id])>=5:
        break

KeyboardInterrupt: ignored

In [ ]:
all_lengths_are_5 = all(len(lst) == 5 for lst in session_dict.values())

# 打印结果
if all_lengths_are_5:
    print("所有列表的长度都为5。")
else:
    print("有一个或多个列表的长度不为5。")


## 做出submission

In [ ]:
# 创建一个新的数据框，其中每个session_id都有其对应的top1到top5
submission_data = {'session_id': [], 'top1': [], 'top2': [], 'top3': [], 'top4': [], 'top5': []}

for session_id, songs in session_dict.items():
    submission_data['session_id'].append(session_id)
    submission_data['top1'].append(songs[0])
    submission_data['top2'].append(songs[1])
    submission_data['top3'].append(songs[2])
    submission_data['top4'].append(songs[3])
    submission_data['top5'].append(songs[4])

# 创建结果数据框
submission = pd.DataFrame(submission_data)

# 拿有規律的以外的剩餘session_id都排名同個字串(使之無意)
###########################################
session_ids_to_replace = set(submission['session_id'])
session_id_list = train_df.loc[~train_df['session_id'].isin(session_ids_to_replace)]['session_id'].drop_duplicates().values


new_data = {'session_id': session_id_list, 'top1': ['00000000000000000000000000000000']*len(session_id_list), 'top2': ['00000000000000000000000000000000']*len(session_id_list), 'top3': ['00000000000000000000000000000000']*len(session_id_list), 'top4': ['00000000000000000000000000000000']*len(session_id_list), 'top5': ['00000000000000000000000000000000']*len(session_id_list)}
new_df = pd.DataFrame(new_data)

# 将原始的 submission 表格和新的数据合并
submission = pd.concat([submission, new_df], ignore_index=True)
submission['session_id'] = submission['session_id'].astype(int)

submission.to_csv('/content/drive/MyDrive/Datagame-2023/test/submission.csv',index=False)

---
#底下為測試用

## >5

### 分割高、中、低密度用戶群

In [6]:
# 去掉<=5的用戶
train_meta_df = train_meta_df[~train_meta_df['session_id'].isin(selected_rows['session_id'])]

# 找出高度集中歌曲的用戶: 前5首歌播放次數加總超過12次
song_id_counts = train_meta_df.groupby('session_id')['song_id'].value_counts()
top_5_song_total = song_id_counts.groupby('session_id').head(5).groupby('session_id').sum()
selected_rows_high_density_list = top_5_song_total[top_5_song_total>12].index.to_list()
selected_rows_high_density = train_meta_df[train_meta_df['session_id'].isin(selected_rows_high_density_list)]

# 找出中度集中歌曲的用戶: 至少重複播放次數2次以上的有3首以上
count_greater_than_two = song_id_counts[song_id_counts >= 2]
song_counts = count_greater_than_two.groupby('session_id').count()
selected_rows_list = song_counts[song_counts>2].index.to_list()
selected_rows_high_density_list = np.array(selected_rows_high_density_list)
selected_rows_list = np.array(selected_rows_list)
mask = np.isin( selected_rows_list, selected_rows_high_density_list, invert=True)
selected_rows_medium_density_list = selected_rows_list[mask]
selected_rows_medium_density = train_meta_df[train_meta_df['session_id'].isin(selected_rows_medium_density_list)]

# 找出低度集中歌曲的用戶
selected_rows_low_density = train_meta_df[~train_meta_df['session_id'].isin(selected_rows_high_density_list) & ~train_meta_df['session_id'].isin(selected_rows_medium_density_list)]

print(len(selected_rows_high_density))
print(len(selected_rows_medium_density))
print(len(selected_rows_low_density))
# unique_counts = train_meta_df.groupby('session_id')['album_id'].nunique()
# under_4_counts = unique_counts[unique_counts <= 4].index.tolist()
# selected_rows = train_meta_df[train_meta_df['session_id'].isin(under_4_counts)]
# selected_rows_sorted = selected_rows.sort_values(['session_id', 'unix_played_at'])
# # 使用 drop_duplicates 保留每個 session_id 中相同 song_id 的最後一行
# final_selected_rows = selected_rows_sorted.drop_duplicates(['session_id', 'song_id'], keep='last')[['session_id','song_id']]
# session_dict = {}

# # 遍历数据框的每一行
# for index, row in final_selected_rows.iterrows():
#     session_id = row['session_id']
#     song_id = row['song_id']

#     # 如果session_id不在字典中，创建一个新的列表
#     if session_id not in session_dict:
#         session_dict[session_id] = []

#     # 将歌曲添加到session_id对应的列表中
#     session_dict[session_id].append(song_id)
#     # 创建一个字典，将 song_id 映射到专辑和艺术家

650680
1766580
8550840


## 預測模型

## 使用 wandb 繪製訓練圖

In [7]:
!pip install wandb
%env WANDB_LOG_MODEL=true
import wandb
wandb.login()
#127b81750f2af55c121c057c14a44d8254de404f

env: WANDB_LOG_MODEL=true


wandb: Currently logged in as: nrnmnrn. Use `wandb login --relogin` to force relogin


True

## 創建新的或使用製作好的 tokenizer (後來發現不用做)

In [ ]:
# #製作新的tokenizer
# train_df = pd.read_parquet(train_source_file_path)
# target_df = pd.read_parquet(train_target_file_path)

# i=0
# text=""
# while i<len(train_df):
#   for j in range(20):
#     text += train_df['song_id'][i+j]+" "
#   text += '\n'
#   i+=20

# i=0
# while i<len(target_df[:]):
#   for j in range(5):
#     text += target_df['song_id'][i+j]+" "
#   text += '\n'
#   i+=5

# file_path = "/content/drive/MyDrive/Datagame-2023/data/source_and_target_songs.txt"
# with open(file_path, "w", encoding="utf-8") as file:
#     file.write(text)

# from tokenizers import (
#     decoders,
#     models,
#     normalizers,
#     pre_tokenizers,
#     processors,
#     trainers,
#     Tokenizer,
# )

# def get_training_corpus():
#   with open(file_path, "r", encoding="utf-8") as file:
#     for line in file:
#       yield line

# tokenizer = Tokenizer(models.BPE())
# tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)
# print(tokenizer.pre_tokenizer.pre_tokenize_str("Let's test pre-tokenization!"))
# trainer = trainers.BpeTrainer(vocab_size=660000, special_tokens=["<|endoftext|>"]) #sorce跟target的歌加起來不重複的歌共650000多首
# tokenizer.train_from_iterator(get_training_corpus(), trainer=trainer)

# tokenizer.model = models.BPE()
# tokenizer.train(["/content/drive/MyDrive/Datagame-2023/data/source_and_target_songs.txt"], trainer=trainer)
# encoding = tokenizer.encode("Let's test this tokenizer.")
# print(encoding.tokens)

# tokenizer.post_processor = processors.ByteLevel(trim_offsets=False)
# sentence = "Let's test this tokenizer."
# encoding = tokenizer.encode(sentence)
# start, end = encoding.offsets[4]
# print(sentence[start:end])

# tokenizer.decoder = decoders.ByteLevel()
# print(tokenizer.decode(encoding.ids))

# tokenizer.save("/content/drive/MyDrive/Datagame-2023/data/source_and_target_songs_tokenizer.json")

# #取用存好的tokenizer使用
# tokenizer = Tokenizer.from_file("/content/drive/MyDrive/Datagame-2023/data/source_and_target_songs_tokenizer.json")
# encoding = tokenizer.encode(df['input'][0])
# tokenizer.decoder = decoders.WordPiece(prefix="##")
# print(tokenizer.decode(encoding))

SyntaxError: ignored

## 載入資料

In [8]:
train_df = pd.read_parquet(train_source_file_path)['song_id']
meta_song_df = pd.read_parquet(meta_song_file_path)['song_id']
total_song_list = pd.concat([train_df, meta_song_df], axis=0)
total_song_list = pd.DataFrame(total_song_list)
total_song_list = total_song_list.drop_duplicates(subset='song_id', keep='first')['song_id'].to_list()
del train_df
del meta_song_df

total_song = ""
for x in total_song_list:
  total_song += x+" "
total_song = total_song[:-1]

In [9]:
total_song_list_size = len(total_song_list)
song_to_int = {total_song_list[i]: i for i in range(total_song_list_size)}
int_to_song = {i: total_song_list[i] for i in range(total_song_list_size)}

In [10]:
target_df = pd.read_parquet(train_target_file_path)
grouped_target = target_df.groupby('session_id')['song_id'].apply(list).reset_index()
del target_df

In [11]:

def make_input_and_label(df):
  grouped_songs  = df[:].groupby('session_id')['song_id'].apply(list).reset_index(name='input')

  # 合并 grouped_songs 和 grouped_target
  grouped_songs = grouped_songs.merge(grouped_target, how='left', on='session_id')
  grouped_songs = grouped_songs.rename(columns={'song_id': 'label'})

  grouped_songs['input'] = grouped_songs['input'].apply(lambda x: ' '.join(x))
  grouped_songs['label'] = grouped_songs['input']+ " "+grouped_songs['label'].apply(lambda x: ' '.join(x))

  grouped_songs = grouped_songs.drop('session_id', axis=1)

  return grouped_songs[:]

In [12]:
df = make_input_and_label(selected_rows_high_density)

### 自製index to song_id，後面發現tokenizer可以解決沒看過的字串的問題

In [13]:
# import pandas as pd
# import torch
# # input_20_songs_sequence = ['song_id_13 song_id_16 song_id_17 song_id_6 song_id_17 song_id_2 song_id_17 song_id_14 song_id_13 song_id_2 song_id_11 song_id_18 song_id_13 song_id_12 song_id_16 song_id_1 song_id_6 song_id_15 song_id_10 song_id_16', 'song_id_18 song_id_19 song_id_5 song_id_7 song_id_17 song_id_12 song_id_6 song_id_15 song_id_19 song_id_1 song_id_20 song_id_19 song_id_17 song_id_19 song_id_19 song_id_8 song_id_8 song_id_5 song_id_3 song_id_7']
# # output_5_songs_sequence = ['song_id_7 song_id_12 song_id_18 song_id_5 song_id_18', 'song_id_6 song_id_19 song_id_8 song_id_15 song_id_14']
# # data = {'input': input_20_songs_sequence, 'label': output_5_songs_sequence}
# # df = pd.DataFrame(data)
# # song_to_int = {f'song_id_{i}': i+1 for i in range(21)}
# # int_to_song = {i+1: f'song_id_{i+1}' for i in range(21)}
# # int_to_song[0] = '[PAD]'

# train_df = pd.read_parquet(train_source_file_path)
# meta_song_df = pd.read_parquet(meta_song_file_path)
# total_song_id = list(set(train_df['song_id'].to_list() + meta_song_df['song_id'].to_list()))
# total_song_size = len(total_song_id)
# del meta_song_df
# del train_df

# song_to_int = {total_song_id[i]: i+1 for i in range(total_song_size)}
# int_to_song = {i+1: total_song_id[i] for i in range(total_song_size)}
# int_to_song[0] = '[PAD]'

## 描述

In [ ]:
#使用Huggingface的生成式模型，並訓練模型
#輸入為使用者聽的前20首歌，預測接下來會聽的5首歌
#預測的5首歌都要不同。若預測的5首歌有重複，則視為預測錯誤
#輸入的20首歌中，有可能有重複的歌曲

## 載入模型

In [14]:
#載入模型
# from transformers import AutoTokenizer, AutoModelForCausalLM, GPT2LMHeadModel, GPT2Config
# import torch

from transformers import GPT2Config, GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
import torch

new_vocab_size = 1100000
config = GPT2Config.from_pretrained("sshleifer/tiny-gpt2")
config.vocab_size = new_vocab_size
# model_checkpoint_path = "/content/drive/MyDrive/Datagame-2023/test/model/medium_density/checkpoint-20500"
# tokenizer = AutoTokenizer.from_pretrained("gpt2")
# model_checkpoint_path = "gpt2"
# model_checkpoint_path = "sshleifer/tiny-gpt2"
# model = GPT2LMHeadModel.from_pretrained(config)
model = GPT2LMHeadModel(config)

device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = model.to(device)

## 載入訓練資料

In [15]:
#載入訓練資料
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
import torch

class SongDataset(Dataset):
    def __init__(self, df):
        self.df = df
        # self.song_to_int = song_to_int
        self.max_len = 700

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        input_20_songs_sequence = self.df.iloc[idx]['input']
        output_5_songs_sequence = self.df.iloc[idx]['label']
        input_20_songs_sequence = [song_to_int[i] for i in (input_20_songs_sequence.split(" "))]
        output_5_songs_sequence = [song_to_int[i] for i in (output_5_songs_sequence.split(" "))]
        input_20_songs_sequence = input_20_songs_sequence
        output_5_songs_sequence = output_5_songs_sequence
        input_20_songs_sequence = input_20_songs_sequence + [0] * (self.max_len - len(input_20_songs_sequence))
        output_5_songs_sequence = output_5_songs_sequence + [0] * (self.max_len - len(output_5_songs_sequence))
        input_20_songs_sequence = torch.tensor(input_20_songs_sequence)
        output_5_songs_sequence = torch.tensor(output_5_songs_sequence)
        return input_20_songs_sequence, output_5_songs_sequence

train_ratio = 0.9999
train_dataset, eval_dataset = train_test_split(df, test_size=(1 - train_ratio), random_state=42, shuffle=True)
# train_dataset = SongDataset(df, tokenizer, song_to_int)
train_dataset = SongDataset(train_dataset)
eval_dataset = SongDataset(eval_dataset)

## 訓練模型


In [16]:
#訓練模型
from transformers import Trainer, TrainingArguments, TrainerCallback
from transformers import default_data_collator

def my_data_collator(features):
    # Your custom data collation logic here
    input_20_songs_sequence, output_5_songs_sequence = zip(*features)

    # Padding sequences to the maximum length
    input_20_songs_sequence = torch.nn.utils.rnn.pad_sequence(input_20_songs_sequence, batch_first=True, padding_value=0)
    output_5_songs_sequence = torch.nn.utils.rnn.pad_sequence(output_5_songs_sequence, batch_first=True, padding_value=0)

    return {
        'input_ids': input_20_songs_sequence,  # Rename to 'input_ids'
        'labels': output_5_songs_sequence,  # Rename to 'labels'
    }

class EarlyStoppingCallback(TrainerCallback):
    def __init__(self, patience=3):
        self.patience = patience
        self.counter = 0
        self.best_loss = float('inf')

    def on_epoch_end(self, args, state, control, **kwargs):
        current_loss = state.log_history[0]["loss"]
        if current_loss < self.best_loss:
            self.best_loss = current_loss
            self.counter = 0
        else:
            self.counter += 1

        if self.counter >= self.patience:
            control.should_training_stop = True

training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/Datagame-2023/test/model/high_density',
    num_train_epochs=1,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_steps=1000,
    save_total_limit=1,
    evaluation_strategy='steps',
    eval_steps=1000,
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',
    greater_is_better=False,
    report_to="wandb"
)

# Update the Trainer instantiation to include the callback
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=my_data_collator,
    callbacks=[EarlyStoppingCallback(patience=3)]
)

trainer.train()

Step,Training Loss,Validation Loss
1000,13.711500,13.710506
2000,13.446800,13.446581
3000,13.152200,13.154541
4000,12.837800,12.840515
5000,12.501400,12.507570
6000,12.159100,12.160275
7000,11.794900,11.801809
8000,11.432000,11.434989
9000,11.056900,11.062218
10000,10.689800,10.686727


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=32530, training_loss=8.888497563973962, metrics={'train_runtime': 2600.7724, 'train_samples_per_second': 12.508, 'train_steps_per_second': 12.508, 'total_flos': 20767152000.0, 'train_loss': 8.888497563973962, 'epoch': 1.0})

## 使用範例

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display

# 指定 Parquet 檔案的路徑
train_source_file_path = '/content/drive/MyDrive/Datagame-2023/data/label_train_source.parquet'
train_target_file_path = '/content/drive/MyDrive/Datagame-2023/data/label_train_target.parquet'
test_source_file_path = '/content/drive/MyDrive/Datagame-2023/data/label_test_source.parquet'
meta_song_file_path = '/content/drive/MyDrive/Datagame-2023/data/meta_song.parquet'
song_composer_file_path = '/content/drive/MyDrive/Datagame-2023/data/meta_song_composer.parquet'
song_genre_file_path = '/content/drive/MyDrive/Datagame-2023/data/meta_song_genre.parquet'
song_lyricist_file_path = '/content/drive/MyDrive/Datagame-2023/data/meta_song_lyricist.parquet'
song_producer_file_path = '/content/drive/MyDrive/Datagame-2023/data/meta_song_producer.parquet'
song_titletext_file_path = '/content/drive/MyDrive/Datagame-2023/data/meta_song_titletext.parquet'


In [6]:
train_df = pd.read_parquet(train_source_file_path)['song_id']
meta_song_df = pd.read_parquet(meta_song_file_path)['song_id']
total_song_list = pd.concat([train_df, meta_song_df], axis=0)
total_song_list = pd.DataFrame(total_song_list)
total_song_list = total_song_list.drop_duplicates(subset='song_id', keep='first')['song_id'].to_list()
del train_df
del meta_song_df

total_song = ""
for x in total_song_list:
  total_song += x+" "
total_song = total_song[:-1]

In [7]:
total_song_list_size = len(total_song_list)
song_to_int = {total_song_list[i]: i for i in range(total_song_list_size)}
int_to_song = {i: total_song_list[i] for i in range(total_song_list_size)}

In [9]:
test_df = pd.read_parquet(test_source_file_path)
meta_song_df = pd.read_parquet(meta_song_file_path)
# 将 NaN 值替换为 0，并转换为整数
meta_song_df['album_id'] = meta_song_df['album_id'].fillna(0).astype(int)
meta_song_df['artist_id'] = meta_song_df['artist_id'].fillna(0).astype(int)

test_meta_df = pd.merge(test_df, meta_song_df[['song_id', 'artist_id', 'album_id']], on='song_id', how='left')
album_song_count = test_meta_df.groupby('album_id')['song_id'].value_counts()
artist_song_count = test_meta_df.groupby('artist_id')['song_id'].value_counts()
song_count = test_meta_df['song_id'].value_counts()

song_id_mapping = dict(zip(meta_song_df['song_id'], zip(meta_song_df['album_id'], meta_song_df['artist_id'])))

del test_df
del meta_song_df

submission_data = {'session_id': [], 'top1': [], 'top2': [], 'top3': [], 'top4': [], 'top5': []}

In [11]:
from transformers import GPT2Config, GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from tqdm import tqdm
import torch

def density_output(df,density, batch_size=16):
  if density == "high":
      model_checkpoint_path = "/content/drive/MyDrive/Datagame-2023/test/model/high_density/checkpoint-32000"
  elif density == 'medium':
      model_checkpoint_path = "/content/drive/MyDrive/Datagame-2023/test/model/medium_density/checkpoint-22500"
  else:
      model_checkpoint_path = "/content/drive/MyDrive/Datagame-2023/test/model/low_density/checkpoint-46000"

  model = GPT2LMHeadModel.from_pretrained(model_checkpoint_path)
  device = "cuda:0" if torch.cuda.is_available() else "cpu"
  model = model.to(device)

  session_ids = df['session_id'].unique()
  num_sessions = len(session_ids)

  for i in tqdm(range(0, num_sessions, batch_size)):
    batch_session_ids = session_ids[i:i+batch_size]

    input_20_songs_list = []
    for session_id in batch_session_ids:
        input_20_songs_list.extend(df[df['session_id'] == session_id]['song_id'].to_list())

    input_20_songs = " ".join(input_20_songs_list)
    input_20_songs = [song_to_int[i] for i in input_20_songs.split(" ")]
    input_20_songs = torch.tensor(input_20_songs).unsqueeze(0).to(model.device)

    output_all_songs = model.generate(
        input_ids=input_20_songs,
        max_length=500,
        num_beams=5,
        no_repeat_ngram_size=2,
        early_stopping=True,
        temperature=0.,
        top_k=2,
        top_p=0.95
    )

    output_all_songs = output_all_songs.tolist()[0]
    output_all_songs = [int_to_song[i] for i in output_all_songs]

    for j, session_id in enumerate(batch_session_ids):
        output_5_songs = output_all_songs[(j*5):(j*5 + 5)]
        output_songs = list(set(output_5_songs))

        k = 5 * j
        while len(output_songs) < 5:
            if output_all_songs[k] not in output_songs:
                output_songs.append(output_all_songs[k])
            k += 1

        submission_data['session_id'].append(session_id)
        submission_data['top1'].append(output_songs[0])
        submission_data['top2'].append(output_songs[1])
        submission_data['top3'].append(output_songs[2])
        submission_data['top4'].append(output_songs[3])
        submission_data['top5'].append(output_songs[4])

In [12]:
# 找出高度集中歌曲的用戶: 前5首歌播放次數加總超過12次
song_id_counts = test_meta_df.groupby('session_id')['song_id'].value_counts()
top_5_song_total = song_id_counts.groupby('session_id').head(5).groupby('session_id').sum()
selected_rows_high_density_list = top_5_song_total[top_5_song_total>12].index.to_list()
selected_rows_high_density = test_meta_df[test_meta_df['session_id'].isin(selected_rows_high_density_list)]
density_output(selected_rows_high_density, "high")

torch.cuda.empty_cache()

  0%|          | 0/887 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:396: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
The attention mask and the pad token id were not set. As a con

In [27]:
test_meta_df = test_meta_df[~test_meta_df['session_id'].isin(selected_rows_high_density['session_id'])]

# 找出中度集中歌曲的用戶: 至少重複播放次數2次以上的有3首以上
count_greater_than_two = song_id_counts[song_id_counts >= 2]
song_counts = count_greater_than_two.groupby('session_id').count()
selected_rows_list = song_counts[song_counts>2].index.to_list()
selected_rows_high_density_list = np.array(selected_rows_high_density_list)
selected_rows_list = np.array(selected_rows_list)
mask = np.isin( selected_rows_list, selected_rows_high_density_list, invert=True)
selected_rows_medium_density_list = selected_rows_list[mask]
selected_rows_medium_density = test_meta_df[test_meta_df['session_id'].isin(selected_rows_medium_density_list)]
density_output(selected_rows_medium_density, "medium")

torch.cuda.empty_cache()

 96%|█████████▌| 1328/1383 [26:52<01:09,  1.26s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
 96%|█████████▌| 1329/1383 [26:53<01:06,  1.23s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
 96%|█████████▌| 1330/1383 [26:54<01:04,  1.21s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
 96%|█████████▌| 1331/1383 [26:55<01:01,  1.19s/it]The attention mask and the pad token id wer

In [28]:
# 找出低度集中歌曲的用戶
selected_rows_low_density = test_meta_df[~test_meta_df['session_id'].isin(selected_rows_medium_density['session_id'])]
density_output(selected_rows_low_density, "low")

torch.cuda.empty_cache()

串流輸出內容已截斷至最後 5000 行。
 63%|██████▎   | 4174/6673 [1:28:01<54:02,  1.30s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
 63%|██████▎   | 4175/6673 [1:28:03<52:43,  1.27s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
 63%|██████▎   | 4176/6673 [1:28:04<51:59,  1.25s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
 63%|██████▎   | 4177/6673 [1:28:05<51:16,  1.23s/it]The attention 

In [29]:
submission = pd.DataFrame(submission_data)
submission['session_id'] = submission['session_id'].astype(int)
submission.to_csv('/content/drive/MyDrive/Datagame-2023/test/submission.csv',index=False)



---



In [23]:
input_20_songs = '6027767fad949f3ca5e772df04924949 041547bddb0a3e730f32db84c65868ca 041547bddb0a3e730f32db84c65868ca 041547bddb0a3e730f32db84c65868ca 8b32f88104ecf859be934d9b45f30cd1 e4a125e3163e4c1bd40060614c79bd53 8b32f88104ecf859be934d9b45f30cd1 5ef6718f4517d2d3c316fc45226f41dc e4a125e3163e4c1bd40060614c79bd53 041547bddb0a3e730f32db84c65868ca e7efab54028017e35a35d1b1637e210c 3f8e8cbe4b5d55f07ba4c7ddfab624b7 3f8e8cbe4b5d55f07ba4c7ddfab624b7 3f8e8cbe4b5d55f07ba4c7ddfab624b7 3f8e8cbe4b5d55f07ba4c7ddfab624b7 3f8e8cbe4b5d55f07ba4c7ddfab624b7 a97177f0f37a2bae91d8e67831949392 6027767fad949f3ca5e772df04924949 6027767fad949f3ca5e772df04924949 6027767fad949f3ca5e772df04924949' #20首歌
# input_20_songs = [song_to_int[song] for song in input_20_songs.split(' ')]
input_20_songs = [song_to_int[i] for i in (input_20_songs.split(" "))]
input_20_songs = torch.tensor(input_20_songs)
input_20_songs = input_20_songs.unsqueeze(0)
input_20_songs = input_20_songs.to(model.device)

output_5_songs = model.generate(input_ids=input_20_songs, max_length=25, num_beams=5, no_repeat_ngram_size=2, early_stopping=True, temperature=0.,  top_k=2, top_p=0.95)
output_5_songs = output_5_songs.tolist()[0]
# output_5_songs = [int_to_song[song] for song in output_5_songs]
output_5_songs = [int_to_song[i] for i in output_5_songs][20:]
print(len(output_5_songs))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


25


In [ ]:
# 將生成的歌曲轉換為歌曲ID的列表
generated_song_ids = [song_to_int[song] for song in input_20_songs]

# 確保生成的5首歌曲都是唯一的
unique_generated_song_ids = list(set(generated_song_ids))

# 如果有歌曲重複，進行調整
while len(unique_generated_song_ids) < 5:
    # 重新生成缺失的歌曲
    missing_count = 5 - len(unique_generated_song_ids)
    missing_songs = model.generate(
        input_ids=input_20_songs_sequence,
        max_length=max_length,
        num_beams=5,
        no_repeat_ngram_size=2,
        early_stopping=True,
        num_return_sequences=missing_count
    ).tolist()

    # 將新生成的歌曲加入列表
    for song_seq in missing_songs:
        song_id = song_seq[-1]  # 取每個序列的最後一個歌曲
        unique_generated_song_ids.append(song_id)

    # 移除重複的歌曲
    unique_generated_song_ids = list(set(unique_generated_song_ids))

# 將歌曲ID轉換為歌曲名稱
unique_generated_songs = [int_to_song[song_id] for song_id in unique_generated_song_ids]

print(unique_generated_songs)


## 其他

### 做出每個session_id對應的album_id跟artist_id的播放數 (session_album_artist)

#### 快速版的

In [ ]:
session_album_artist = {}
# 遍历每个 session_id 的 song_id
for session_id, song_id in zip(selected_rows['session_id'], selected_rows['song_id']):
    if song_id not in song_id_mapping:
        continue

    # 获取专辑和艺术家
    album_id, artist_id = song_id_mapping[song_id]

    # 获取局部变量以减少嵌套访问
    session_albums = session_album_artist.setdefault(session_id, {'albums': {}})
    session_artists = session_albums.setdefault('artists', {})

    # album_id 非 nan
    if not np.isnan(album_id):
        album_id = int(album_id)
        session_albums['albums'][album_id] = session_albums['albums'].get(album_id, 0) + 1

    # artist_id 非 nan
    if not np.isnan(artist_id):
        artist_id = int(artist_id)
        session_artists[artist_id] = session_artists.get(artist_id, 0) + 1

    session_album_artist[session_id]['albums'] = dict(sorted(session_album_artist[session_id]['albums'].items(), key=lambda x: x[1], reverse=True))
    session_album_artist[session_id]['artists'] = dict(sorted(session_album_artist[session_id]['artists'].items(), key=lambda x: x[1], reverse=True))

#### 慢速版的

In [ ]:
# result_dict = {}

# # 遍历每个session_id的song_id
# for session_id, song_id in zip(selected_rows['session_id'][2000:2100], selected_rows['song_id'][2000:2100]):
#     if song_id not in meta_song_df['song_id'].values:
#       # 可以选择忽略该条记录或者采取其他处理方式
#       continue

#     # 找到对应的专辑和艺术家
#     album_id = meta_song_df[meta_song_df['song_id']==song_id]['album_id'].values[0]
#     artist_id = meta_song_df[meta_song_df['song_id']==song_id]['artist_id'].values[0]

#     # 更新字典
#     if session_id not in result_dict:
#         result_dict[session_id] = {'albums': {}, 'artists': {}}

#     # album_id非nan
#     if not np.isnan(album_id):
#       album_id = album_id.astype(int)
#       # 统计专辑数量
#       if album_id not in result_dict[session_id]['albums']:

#           result_dict[session_id]['albums'][album_id] = 1
#       else:
#           result_dict[session_id]['albums'][album_id] += 1

#     # artist_id非nan
#     if not np.isnan(artist_id):
#       artist_id = artist_id.astype(int)
#       # 统计艺术家数量
#       if artist_id not in result_dict[session_id]['artists'] and not np.isnan(artist_id):
#           result_dict[session_id]['artists'][artist_id] = 1
#       else:
#           result_dict[session_id]['artists'][artist_id] += 1

# # 输出结果
# print(result_dict)


### 找循環以及填補5個剩餘的空缺

In [ ]:
session_ids = train_meta_df['session_id'].unique()
for session_id in session_ids:
  # 从最后一行开始往上追踪
  song_20 = train_meta_df[train_meta_df['session_id']==session_id].drop_duplicates(['session_id', 'song_id'], keep='last')['song_id'].values
  cycle_song = [song_20[len(song_20)-1]]
  # 從最後開始往前找
  for i in range(len(song_20)-2,-1,-1):
    song_id = song_20[i]
    # cycle_song不能有重複的歌曲
    if song_id not in cycle_song:
      cycle_song.append(song_id)
    # 如果後面遇到跟最後一首歌一樣的話，開始尋找current_match的最長max_match
    elif song_id == cycle_song[0]:
      break
  max_match = []
  current_match = []
  b = len(song_20)-1-len(cycle_song) # song_20的位置
  a = 0 # cycle_song的位置

  while b > -1:
    while True:
      if song_20[b] == cycle_song[a] and a < len(cycle_song):
        current_match.append(song_20[b])
        b -= 1
        a += 1
      else:
        if len(current_match)>len(max_match):
          max_match = current_match.copy()
          current_match = []
          a = 0
        break
    b -= 1
  if len(max_match)>1:
    for song_id in reversed(cycle_song):
      if len(session_dict[session_id])<5:
        session_dict[session_id].append(song_id)
      else:
        break
    session_ids.remove(session_id)

# 剩下是沒補完5個空缺的填補方式
  # 找歌曲重複播放次數>=三次的
  # session_id重複播放歌曲的次數
  for song_id, count in selected_rows[selected_rows['session_id']==session_id]['song_id'].value_counts().iteritems():
    if len(session_dict[session_id])==5 or count<3:
      break
    else:
      session_dict[session_id].append(song_id)
  # 透過album填補
  if len(session_dict[session_id])<5:
    if not session_album_artist[session_id]['albums']=={}:
      albums = session_album_artist[session_id]['albums']
      for album in albums:
        # 此album播放的次數必須大於3，代表此用戶在聽此album
        if albums[album]>3:
          # 抓出專輯裡的所有歌曲由高到低排名
          for song_id in album_song_count.loc[album].index.get_level_values('song_id').tolist():
            if song_id not in session_dict[session_id]:
              session_dict[session_id].append(song_id)
            if len(session_dict[session_id])>=5:
              break
        else:
          break
        if len(session_dict[session_id])>=5:
          break
  else:
    continue
  # 透過artist填補
  if len(session_dict[session_id])<5:
    if not session_album_artist[session_id]['artists']=={}:
      artists = session_album_artist[session_id]['artists']
      for artist in artists:
        # 此artists播放的次數必須大於3，代表此用戶在聽此artists
        if artists[artist]>3:
          # 抓出artists的所有歌曲由高到低排名
          for song_id in artist_song_count.loc[artist].index.get_level_values('song_id').tolist():
            if song_id not in session_dict[session_id]:
              session_dict[session_id].append(song_id)
            if len(session_dict[session_id])>=5:
              break
        else:
          break
        if len(session_dict[session_id])>=5:
          break
  else:
    continue
  # 從最紅的歌開始填補
  if len(session_dict[session_id])<5:
    for song_id in song_count.index:
      if song_id not in session_dict[session_id]:
        session_dict[session_id].append(song_id)
      if len(session_dict[session_id])>=5:
        break


In [ ]:
# 创建一个新的数据框，其中每个session_id都有其对应的top1到top5
submission_data = {'session_id': [], 'top1': [], 'top2': [], 'top3': [], 'top4': [], 'top5': []}

for session_id, songs in session_dict.items():
    submission_data['session_id'].append(session_id)
    submission_data['top1'].append(songs[0])
    submission_data['top2'].append(songs[1])
    submission_data['top3'].append(songs[2])
    submission_data['top4'].append(songs[3])
    submission_data['top5'].append(songs[4])

# 创建结果数据框
submission = pd.DataFrame(submission_data)

# 拿有規律的以外的剩餘session_id都排名同個字串(使之無意)
###########################################
session_ids_to_replace = set(submission['session_id'])
session_id_list = train_df.loc[~train_df['session_id'].isin(session_ids_to_replace)]['session_id'].drop_duplicates().values


new_data = {'session_id': session_id_list, 'top1': ['00000000000000000000000000000000']*len(session_id_list), 'top2': ['00000000000000000000000000000000']*len(session_id_list), 'top3': ['00000000000000000000000000000000']*len(session_id_list), 'top4': ['00000000000000000000000000000000']*len(session_id_list), 'top5': ['00000000000000000000000000000000']*len(session_id_list)}
new_df = pd.DataFrame(new_data)

# 将原始的 submission 表格和新的数据合并
submission = pd.concat([submission, new_df], ignore_index=True)
submission['session_id'] = submission['session_id'].astype(int)

submission.to_csv('/content/drive/MyDrive/Datagame-2023/test/submission.csv',index=False)

## 播放過的歌曲數量為5


In [ ]:
train_df = pd.read_parquet(test_source_file_path)
train_df = train_df[:]

In [ ]:
unique_counts = train_df.groupby('session_id')['song_id'].nunique()

# 找出唯一數量為 5 的 session_id
unique_5_counts = unique_counts[unique_counts == 5].index.tolist()
selected_rows = train_df[train_df['session_id'].isin(unique_5_counts)]
selected_rows_sorted = selected_rows.sort_values(['session_id', 'unix_played_at'])
# 使用 drop_duplicates 保留每個 session_id 中相同 song_id 的最後一行
final_selected_rows = selected_rows_sorted.drop_duplicates(['session_id', 'song_id'], keep='last')[['session_id','song_id']]

In [ ]:
session_dict = {}

# 遍历数据框的每一行
for index, row in final_selected_rows.iterrows():
    session_id = row['session_id']
    song_id = row['song_id']

    # 如果session_id不在字典中，创建一个新的列表
    if session_id not in session_dict:
        session_dict[session_id] = []

    # 将歌曲添加到session_id对应的列表中
    session_dict[session_id].append(song_id)

# 创建一个新的数据框，其中每个session_id都有其对应的top1到top5
submission_data = {'session_id': [], 'top1': [], 'top2': [], 'top3': [], 'top4': [], 'top5': []}

for session_id, songs in session_dict.items():
    submission_data['session_id'].append(session_id)
    submission_data['top1'].append(songs[0])
    submission_data['top2'].append(songs[1])
    submission_data['top3'].append(songs[2])
    submission_data['top4'].append(songs[3])
    submission_data['top5'].append(songs[4])

# 创建结果数据框
submission = pd.DataFrame(submission_data)

# 拿有規律的以外的剩餘session_id的排名都為無義字串
###########################################
session_ids_to_replace = set(submission['session_id'])
session_id_list = train_df.loc[~train_df['session_id'].isin(session_ids_to_replace)]['session_id'].drop_duplicates().values


new_data = {'session_id': session_id_list, 'top1': ['00000000000000000000000000000000']*len(session_id_list), 'top2': ['00000000000000000000000000000000']*len(session_id_list), 'top3': ['00000000000000000000000000000000']*len(session_id_list), 'top4': ['00000000000000000000000000000000']*len(session_id_list), 'top5': ['00000000000000000000000000000000']*len(session_id_list)}
new_df = pd.DataFrame(new_data)

# 将原始的 submission 表格和新的数据合并
submission = pd.concat([submission, new_df], ignore_index=True)
submission['session_id'] = submission['session_id'].astype(int)

submission.to_csv('/content/drive/MyDrive/Datagame-2023/test/submission.csv',index=False)

## 找循環

In [ ]:
import pandas as pd
import numpy as np

A = [1, 2, 3, 1, 2, 3, 1, 2, 3]

# 创建 Pandas 数据框
df = pd.DataFrame({'value': A})

# 找出相邻重复元素的索引
duplicate_indices = df[df['value'].eq(df['value'].shift(-3))].index

# 计算重复元素之间的距离
distances = np.diff(duplicate_indices)

print(distances)


[1 1 1 1 1]


In [ ]:
import pandas as pd

A = [1, 2, 3, 1, 2, 3, 1, 2, 3]

# 创建 Pandas 数据框
df = pd.DataFrame({'value': A})

# 找到和最后一个元素相同的前一个元素的位置
last_value = df['value'].iloc[-1]
last_value_index = df[df['value'].eq(last_value)].index[-2]

print(last_value_index)


5
